In [22]:
import findspark
findspark.init()

In [2]:
import pyspark
import random
#sc = pyspark.SparkContext(appName="Pi")

In [23]:
from pyspark.sql import SparkSession
spark = SparkSession \
                .builder \
                .appName("als") \
                .config("spark.some.config.option", "some-value") \
                .getOrCreate()

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

train = spark.read.text("train_must.txt").rdd
trainParts = train.map(lambda row: row.value.split("::"))
#column type must be integer
trainRatingsRDD = trainParts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2])))#, timestamp=int(p[3])))
trainRatings = spark.createDataFrame(trainRatingsRDD)

test = spark.read.text("test.txt").rdd
testParts = test.map(lambda row: row.value.split("::"))
#column type must be integer
testRatingsRDD = testParts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2])))#, timestamp=int(p[3])))
testRatings = spark.createDataFrame(testRatingsRDD)


(moreTrain, valid, test) = testRatings.randomSplit([0.6, 0.2, 0.2])

In [7]:
train = trainRatings.union(moreTrain)

In [8]:


# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank = 70, maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(train)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(train)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Training Root-mean-square error = " + str(rmse))
predictions = model.transform(valid)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Validation Root-mean-square error = " + str(rmse))
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Test Root-mean-square error = " + str(rmse))


# Generate top 10 movie recommendations for each user
#userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
#movieRecs = model.recommendForAllItems(10)

Training Root-mean-square error = 0.10611870683192627
Validation Root-mean-square error = 0.491230902111566
Test Root-mean-square error = 0.4899083222787495


In [9]:


# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank = 70, maxIter=20, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(train)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(train)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Training Root-mean-square error = " + str(rmse))
predictions = model.transform(valid)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Validation Root-mean-square error = " + str(rmse))
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Test Root-mean-square error = " + str(rmse))


# Generate top 10 movie recommendations for each user
#userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
#movieRecs = model.recommendForAllItems(10)

Training Root-mean-square error = 0.1016587743933676
Validation Root-mean-square error = 0.31840194714853726
Test Root-mean-square error = 0.31789103189268225


In [13]:
for k in [10, 25, 50, 70, 75, 100, 125, 150]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
    for lam in [0.1]:
        # Build the recommendation model using ALS on the training data
        # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
        als = ALS(rank = k, maxIter=20, regParam=lam, userCol="userId", itemCol="movieId", ratingCol="rating",
                  coldStartStrategy="drop")
        model = als.fit(train)

        # Evaluate the model by computing the RMSE on the test data
        predictions = model.transform(train)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)
        print("Training Root-mean-square error = " + str(rmse))
        predictions = model.transform(valid)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)
        print("Validation Root-mean-square error = " + str(rmse))
        predictions = model.transform(test)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)
        print("Test Root-mean-square error = " + str(rmse))


        # Generate top 10 movie recommendations for each user
        #userRecs = model.recommendForAllUsers(10)
        # Generate top 10 user recommendations for each movie
        #movieRecs = model.recommendForAllItems(10)

Training Root-mean-square error = 0.10226729278244448
Validation Root-mean-square error = 0.3512015927875305
Test Root-mean-square error = 0.35000201317840857
Training Root-mean-square error = 0.10196893234459056
Validation Root-mean-square error = 0.3326656058697553
Test Root-mean-square error = 0.3334197974954108
Training Root-mean-square error = 0.10169339200288229
Validation Root-mean-square error = 0.31924523048077486
Test Root-mean-square error = 0.3203673327118611
Training Root-mean-square error = 0.10165877424114217
Validation Root-mean-square error = 0.3184019470916302
Test Root-mean-square error = 0.3178910319389154
Training Root-mean-square error = 0.10164830542867592
Validation Root-mean-square error = 0.3179055344730976
Test Root-mean-square error = 0.31703223875918124
Training Root-mean-square error = 0.10181451301192777
Validation Root-mean-square error = 0.3253704558846819
Test Root-mean-square error = 0.32456654855057004
Training Root-mean-square error = 0.101719916659

In [14]:
model.rank

150

In [15]:
als.alpha

Param(parent='ALS_4abfbb9382b377e07270', name='alpha', doc='alpha for implicit preference')

In [36]:
als.getAlpha()

1.0

In [24]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

train = spark.read.text("train_must.txt").rdd
trainParts = train.map(lambda row: row.value.split("::"))
#column type must be integer
trainRatingsRDD = trainParts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2])))#, timestamp=int(p[3])))
trainRatings = spark.createDataFrame(trainRatingsRDD)

test = spark.read.text("test.txt").rdd
testParts = test.map(lambda row: row.value.split("::"))
#column type must be integer
testRatingsRDD = testParts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2])))#, timestamp=int(p[3])))
testRatings = spark.createDataFrame(testRatingsRDD)


(moreTrain, valid, test) = testRatings.randomSplit([0.6, 0.2, 0.2])
train = trainRatings.union(moreTrain)

In [25]:
als = ALS(rank = 75, maxIter=20, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
                  coldStartStrategy="drop")
model = als.fit(train)

# Evaluate the model by computing the RMSE on the test data
#predictions = model.transform(train)
#evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                        predictionCol="prediction")
#rmse = evaluator.evaluate(predictions)
#print("Training Root-mean-square error = " + str(rmse))
#predictions = model.transform(valid)
#evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                        predictionCol="prediction")
#rmse = evaluator.evaluate(predictions)
#print("Validation Root-mean-square error = " + str(rmse))
#predictions = model.transform(test)
#evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                        predictionCol="prediction")
#rmse = evaluator.evaluate(predictions)
#print("Test Root-mean-square error = " + str(rmse))

In [26]:
# Generate top 10 movie recommendations for each user
top10 = model.recommendForAllUsers(10)

In [28]:
top10.show()

+-------+--------------------+
| userId|     recommendations|
+-------+--------------------+
|3000113|[[1448342,0.90469...|
|3000375|[[1683427,0.90797...|
|3001312|[[219272,0.908586...|
|3001517|[[308256,0.908835...|
|3001633|[[1534758,0.91522...|
|3001877|[[1355308,0.90807...|
|3001909|[[1532048,0.89999...|
|3002485|[[784336,0.903293...|
|3002702|[[1115935,0.93320...|
|3003719|[[1847349,0.89999...|
|3003804|[[551365,0.904524...|
|3003922|[[2584911,0.89999...|
|3004604|[[1983367,0.90677...|
|3004706|[[674799,0.901197...|
|3005540|[[2612908,0.89999...|
|3005815|[[232518,0.914768...|
|3005949|[[2107077,0.90509...|
|3006039|[[1092531,0.89999...|
|3006068|[[1203999,0.90057...|
|3006917|[[1115935,0.92872...|
+-------+--------------------+
only showing top 20 rows



In [33]:
top10.select(format_string('%d\t%s',top10.userId, top10.recommendations)).write.save("top10rec.out",format="text")

In [34]:
for i in [50]:
    topk = model.recommendForAllUsers(i)
    topk.select(format_string('%d\t%s',topk.userId, topk.recommendations)).write.save("top{}rec.out".format(i),format="text")

In [96]:
result = dict()

In [97]:
with open('../top50rec') as f:
    line = f.readline()
    while line:
        a = int(line[:7])
        b = eval(line[8:])
        result[a] = b
        line = f.readline()
        #break


In [42]:
len(result)

47018

In [89]:
#trainDF = trainRatingsRDD.collect()
trainDF = train.collect()

In [90]:
trainDF[0].movieId

241938

In [91]:
trainDF[0].userId

3020287

In [92]:
from collections import defaultdict
trainDict = defaultdict(list)

In [93]:
for row in trainDF:
    trainDict[row.userId].append(row.movieId)

In [94]:
len(trainDict)

47018

In [53]:
for row in trainDF:
    print(type(row.userId))
    break

<class 'int'>


In [95]:
removeDict = defaultdict(list)
for key, values in result.items():
    for value in values:
        if value[0] in trainDict[key]:
            removeDict[key].append(value)

In [98]:
for key, values in removeDict.items():
    for value in values:
        result[key].remove(value)

In [99]:
for key, value in result.items():
    print(len(value))
    break

50


In [63]:
import pickle
with open('recommendation_minus_train.pickle', 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [100]:
testDF = test.collect()

In [101]:
testDF[0]

Row(movieId=317, rating=1.0, userId=3045521)

In [102]:
testDict = defaultdict(list)
for row in testDF:
    testDict[row.userId].append(row.movieId)

In [103]:
precisions, recalls = [], []
for key, values in result.items():
    n_rel = len(testDict[key])
    n_rec_k = len(values)
    n_rel_and_rec_k = 0
    for value in values:
        if value[0] in testDict[key]:
            n_rel_and_rec_k += 1
    precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
    recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )

In [104]:
import numpy as np
np.sum(precisions)/len(precisions)

0.004734358568840821

In [105]:
np.sum(recalls)/len(recalls)

0.6142697455697159

In [106]:
count = 0
for key, value in result.items():
    if len(value) == 0:
        count += 1
        
count

0

In [107]:
print("f1 score at k=50: ", 2*(0.004734358568840821*0.6142697455697159)/(0.004734358568840821+0.6142697455697159))

f1 score at k=50:  0.009396297097463816


In [108]:
top40dict = dict()
for key, value in result.items():
    if len(value) >= 40:
        top40dict[key]=value[:40]
    else:
        top40dict[key]=value
        


In [109]:
precisions, recalls = [], []
for key, values in top40dict.items():
    n_rel = len(testDict[key])
    n_rec_k = len(values)
    n_rel_and_rec_k = 0
    for value in values:
        if value[0] in testDict[key]:
            n_rel_and_rec_k += 1
    precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
    recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )
print("precision: ",np.sum(precisions)/len(precisions))
print("recall: ",np.sum(recalls)/len(recalls))

precision:  0.0049686529038422056
recall:  0.6065296126170603


In [110]:
print("f1 score at k=40: ", 2*( 0.0049686529038422056*0.6065296126170603)/( 0.0049686529038422056+0.6065296126170603))

f1 score at k=40:  0.009856561468506836


In [76]:
for k in [35, 30, 25, 20, 15, 10, 5, 4, 3, 2, 1]:
    topDict = dict()
    for key, value in result.items():
        if len(value) >= k:
            topDict[key]=value[:k]
        else:
            topDict[key]=value
    precisions, recalls = [], []
    for key, values in topDict.items():
        n_rel = len(testDict[key])
        n_rec_k = len(values)
        n_rel_and_rec_k = 0
        for value in values:
            if value[0] in testDict[key]:
                n_rel_and_rec_k += 1
        precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
        recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )
    precision=np.sum(precisions)/len(precisions)
    recall=np.sum(recalls)/len(recalls)
    print("precision at k={}: ".format(k),precision)
    print("recall at k={}: ".format(k),recall)
    print("f1 score at k={}".format(k),2*(precision*recall)/(precision+recall))

precision at k=35:  0.0762903772202253
recall at k=35:  0.6471606271382133
precision at k=30:  0.0838333341301656
recall at k=30:  0.6376226239712638
precision at k=25:  0.09446362272744703
recall at k=25:  0.6256170967987298
precision at k=20:  0.10976820060747242
recall at k=20:  0.6107585849545819
precision at k=15:  0.13306650216184862
recall at k=15:  0.5913025555879567
precision at k=10:  0.17286008161683436
recall at k=10:  0.5629213601067565
precision at k=5:  0.25571164234973837
recall at k=5:  0.5088729134924429
precision at k=4:  0.28484835594878555
recall at k=4:  0.49025417372928504
precision at k=3:  0.3220362414394487
recall at k=3:  0.46416437115442954
precision at k=2:  0.37259134799438515
recall at k=2:  0.4261272678557348
precision at k=1:  0.44363860649113107
recall at k=1:  0.35937701596777377


In [188]:
for k in [50,40,35, 30, 25, 20, 15, 10, 5, 4, 3, 2, 1]:
    topDict = dict()
    for key, value in result.items():
        if len(value) >= k:
            topDict[key]=value[:k]
        else:
            topDict[key]=value
    precisions, recalls = [], []
    for key, values in topDict.items():
        n_rel = len(testDict[key])
        if n_rel == 0:
            continue
        n_rec_k = len(values)
        n_rel_and_rec_k = 0
        for value in values:
            if value[0] in testDict[key]:
                n_rel_and_rec_k += 1
        precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
        recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )
    precision=np.sum(precisions)/len(precisions)
    recall=np.sum(recalls)/len(recalls)
    print("precision at k={}: ".format(k),precision)
    print("recall at k={}: ".format(k),recall)
    print("f1 score at k={}".format(k),2*(precision*recall)/(precision+recall))

precision at k=50:  0.00972774859894934
recall at k=50:  0.20743499091888756
f1 score at k=50 0.018583993246397014
precision at k=40:  0.010209156239691161
recall at k=40:  0.19153123830043886
f1 score at k=40 0.019385035317770624
precision at k=35:  0.01068244002919223
recall at k=35:  0.1771568202136579
f1 score at k=35 0.02014985690689042
precision at k=30:  0.011201271130679652
recall at k=30:  0.16031131781031488
f1 score at k=30 0.020939460446575874
precision at k=25:  0.01179237862168422
recall at k=25:  0.14119454430305978
f1 score at k=25 0.021766821554504885
precision at k=20:  0.012317003889350173
recall at k=20:  0.11871384584619997
f1 score at k=20 0.022318391492666186
precision at k=15:  0.01272560416029367
recall at k=15:  0.09188115592452994
f1 score at k=15 0.022355022163723887
precision at k=10:  0.01345977363107984
recall at k=10:  0.06470313754337756
f1 score at k=10 0.022283959782682213
precision at k=5:  0.01451732727352183
recall at k=5:  0.03369253850342686
f1 s

In [112]:
uniq = set()
for key, value in topDict.items():
    for each in value:
        uniq.add(each[0])
len(uniq)

32272

In [113]:
uniq = set()
for key, value in result.items():
    for each in value:
        uniq.add(each[0])
len(uniq)

306514

In [126]:
list(uniq)[0]

2621447

In [85]:
import itertools


[[2353240, 0.35424072], [810284, 0.35424072]]


In [114]:
validDF = valid.collect()

In [115]:
validDict = defaultdict(list)
for row in validDF:
    validDict[row.userId].append(row.movieId)

In [116]:
precisions, recalls = [], []
for key, values in result.items():
    n_rel = len(validDict[key])
    n_rec_k = len(values)
    n_rel_and_rec_k = 0
    for value in values:
        if value[0] in validDict[key]:
            n_rel_and_rec_k += 1
    precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
    recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )

In [117]:
print("precision: ",np.sum(precisions)/len(precisions))
print("recall: ",np.sum(recalls)/len(recalls))

precision:  0.0047157290147397295
recall:  0.611469483437209


In [118]:
print("f1 score at k=50: ", 2*(0.0047157290147397295*0.611469483437209)/(0.0047157290147397295+0.611469483437209))

f1 score at k=50:  0.0093592781079524


In [187]:
for k in [50, 40, 35, 30, 25, 20, 15, 10, 5, 4, 3, 2, 1]:
    topDict = dict()
    for key, value in result.items():
        if len(value) >= k:
            topDict[key]=value[:k]
        else:
            topDict[key]=value
    precisions, recalls = [], []
    for key, values in topDict.items():
        n_rel = len(validDict[key])
        if n_rel == 0:
            continue
        n_rec_k = len(values)
        n_rel_and_rec_k = 0
        for value in values:
            if value[0] in validDict[key]:
                n_rel_and_rec_k += 1
        precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
        recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )
    precision=np.sum(precisions)/len(precisions)
    recall=np.sum(recalls)/len(recalls)
    print("precision at k={}: ".format(k),precision)
    print("recall at k={}: ".format(k),recall)
    print("f1 score at k={}".format(k),2*(precision*recall)/(precision+recall))

precision at k=50:  0.009643115157440638
recall at k=50:  0.20550046415216341
f1 score at k=50 0.01842178741365164
precision at k=40:  0.010116459978234665
recall at k=40:  0.18960837760270288
f1 score at k=40 0.01920808234757905
precision at k=35:  0.010592104614630742
recall at k=35:  0.17610811224771705
f1 score at k=35 0.019982360810949246
precision at k=30:  0.011112762894414582
recall at k=30:  0.15974212795220305
f1 score at k=30 0.020779930657948868
precision at k=25:  0.011601936821351425
recall at k=25:  0.14020907157315007
f1 score at k=25 0.021430551148765638
precision at k=20:  0.012186317574914105
recall at k=20:  0.11807619471274747
f1 score at k=20 0.022092526568646995
precision at k=15:  0.01281259513765059
recall at k=15:  0.09409729628619104
f1 score at k=15 0.02255414433231113
precision at k=10:  0.01340407950245727
recall at k=10:  0.06563810817621439
f1 score at k=10 0.022261995681637083
precision at k=5:  0.01433479754707955
recall at k=5:  0.03467845225263083
f1

In [120]:
len(validDF)

50452

In [121]:
len(trainDF)

500925

In [122]:
len(testDF)

51029

In [128]:
for key, value in topDict.items():
    print(key, value)
    break

3014656 [[2844431, 0.89999855]]


In [129]:
import sys
sys.path.insert(0, '../src')

import preprocess

In [130]:
all_dict = preprocess.create_paper_paper_dict(debug=True)

In [131]:
# This data contains papers which cites 0 or 1 papers

def filter_0_or_more(data_dict):
  # papers which cite 0 paper is useless. remove from data.
    count0_list = []
    for key, value in data_dict.items():
        if len(value) == 0:
            count0_list.append(key)
    for each in count0_list:
        del data_dict[each]
    return count0_list, data_dict

count0_list, rest_dict = filter_0_or_more(all_dict)
len(rest_dict)

47018

In [132]:
popular_result = defaultdict(int)
for value in rest_dict.values():
    for paper in value:
        popular_result[paper] += 1

In [134]:
import operator
sorted_popular = sorted(popular_result.items(), key=operator.itemgetter(1))
sorted_popular.reverse()

In [135]:
sorted_popular

[('e2f7a74a-8430-4463-94ce-fe85dfd309f9', 587),
 ('153c5014-dc7a-44a8-a93f-5cd27f1193df', 450),
 ('c1b6b493-01ef-420f-be44-7bacfe34e846', 402),
 ('f6bd8b64-684d-429a-aab5-8ff3a2c23cd6', 388),
 ('b944f77f-113b-4a02-ae5e-d4a124b8fd5b', 379),
 ('dd83785a-dd19-41e3-9b25-ebabbd48d336', 345),
 ('546cc930-3d5a-4208-a77b-a506f146ab97', 324),
 ('bff1945c-7b01-4b42-b6c4-1e3601c18a6b', 302),
 ('c93eac1a-7d9a-48ab-9fb4-389c85bea00e', 281),
 ('3715c870-6569-4d20-8797-af8569e5fde5', 242),
 ('051956bb-f64b-4fdb-87f8-3e2868b8b5d8', 242),
 ('8026f56a-a93e-4933-8ead-c9aa9e3f0498', 241),
 ('c186e8f6-42e1-4bb8-8fe3-039e0cd02532', 240),
 ('65d5ccdc-7022-45b0-adf9-0385273b1283', 226),
 ('7f1214b2-e070-4ff2-a5d3-647e7c16c2d7', 210),
 ('2b6a3d0f-368f-45bb-be23-4e82f62fbbf7', 209),
 ('176a7436-78ea-4c2a-82e6-7930ab023bd1', 202),
 ('493f502b-b1b8-412c-95fd-3c1103480f1d', 199),
 ('12d6aa75-3066-4e5f-a73d-f0d56c9d99f5', 193),
 ('12f40b38-cd99-4801-8074-d765a29a2101', 192),
 ('50dd56db-151d-4d62-8576-65f0ef6f381b'

In [140]:
def assign_number_to_paper_id(datadir='../dblp-ref'):
    # When using surprise, it seems like we don't need this method.
    # This method assigns nonnegative integer numbers to each paper id,
    # and record it in a dictionary for efficient lookup.
    DBLP_LIST = [ datadir+'/dblp-ref-0.json',
        datadir+'/dblp-ref-1.json',
        datadir+'/dblp-ref-2.json',
        datadir+'/dblp-ref-3.json' ]
    
    numbering, reverse = dict(), dict()
    current_id = 0

    for data in DBLP_LIST:
        with open(data) as f:
            line = f.readline()
            while line:
                data = json.loads(line)
                numbering[data["id"]] = current_id
                reverse[current_id]   = data["id"]
                current_id += 1
                line = f.readline()

    return numbering, reverse

In [142]:
import json
numbering, reverse = assign_number_to_paper_id()

In [143]:
popular_result = defaultdict(int)
for value in rest_dict.values():
    for paper in value:
        popular_result[numbering[paper]] += 1
sorted_popular = sorted(popular_result.items(), key=operator.itemgetter(1))
sorted_popular.reverse()
sorted_popular

[(2458466, 587),
 (566845, 450),
 (2358174, 402),
 (2716671, 388),
 (2332760, 379),
 (2442067, 345),
 (337046, 324),
 (2570279, 302),
 (2381049, 281),
 (2947204, 242),
 (1789325, 242),
 (2160922, 241),
 (2357628, 240),
 (2081801, 226),
 (2157629, 210),
 (1905044, 209),
 (1844720, 202),
 (1995533, 199),
 (1831094, 193),
 (543450, 192),
 (2018485, 191),
 (2687881, 183),
 (612667, 179),
 (648142, 177),
 (2465088, 151),
 (2190418, 147),
 (2118917, 145),
 (2455911, 143),
 (1774946, 143),
 (1436324, 142),
 (1688452, 142),
 (2156647, 137),
 (563278, 129),
 (593467, 127),
 (209083, 120),
 (2016464, 117),
 (2064561, 116),
 (439498, 116),
 (2297773, 116),
 (2130367, 116),
 (2468314, 112),
 (2612755, 111),
 (1580227, 111),
 (1431819, 110),
 (619580, 109),
 (509302, 108),
 (1930047, 108),
 (1244162, 107),
 (2015202, 106),
 (2195345, 106),
 (2505760, 104),
 (2567800, 103),
 (1983367, 103),
 (1397305, 102),
 (2140646, 102),
 (564593, 100),
 (564527, 98),
 (1822381, 96),
 (2006147, 96),
 (2002755, 96

In [147]:
sorted_popular_numbers_only = []
i = 0
for each in sorted_popular:
    sorted_popular_numbers_only.append(each[0])
    i += 1
    if i > 50:
        break

In [150]:
sorted_popular_numbers_only

[2458466,
 566845,
 2358174,
 2716671,
 2332760,
 2442067,
 337046,
 2570279,
 2381049,
 2947204,
 1789325,
 2160922,
 2357628,
 2081801,
 2157629,
 1905044,
 1844720,
 1995533,
 1831094,
 543450,
 2018485,
 2687881,
 612667,
 648142,
 2465088,
 2190418,
 2118917,
 2455911,
 1774946,
 1436324,
 1688452,
 2156647,
 563278,
 593467,
 209083,
 2016464,
 2064561,
 439498,
 2297773,
 2130367,
 2468314,
 2612755,
 1580227,
 1431819,
 619580,
 509302,
 1930047,
 1244162,
 2015202,
 2195345,
 2505760]

In [159]:
total_confidence_score = 0
for key, values in topDict.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:1] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            confidence_score += confidence
    total_confidence_score += confidence_score/len(values)
total_confidence_score/len(topDict)

0.5931628216062148

In [160]:
total_confidence_score = 0
for key, values in topDict.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:1] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            confidence_score += 1
    total_confidence_score += confidence_score/len(values)
total_confidence_score/len(topDict)

0.657599217321026

In [158]:
total_confidence_score = 0
for key, values in result.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:50] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            #print(value)
            confidence_score += 1
    total_confidence_score += confidence_score/len(values)
total_confidence_score/len(result)

0.14397271272323972

In [162]:
a, b = 0, 0
total_confidence_score = 0
for key, values in result.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:50] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            #print(value)
            confidence_score += confidence
        if value in sorted_popular_numbers_only[:50]:
            a+=1
        else:
            b+=1
    total_confidence_score += confidence_score/len(values)
print(a, b)
total_confidence_score/len(result)

25927 2220304


0.12934912792854478

In [127]:
#takes way too long!
'''
numEquals, count = 0, 0
for pair in itertools.combinations(result.keys(), 2):
    numEquals += topDict[pair[0]][0]==topDict[pair[1]][0]
    count += 1
numEquals/count
'''

KeyboardInterrupt: 

In [163]:
popular_result = defaultdict(int)
for row in trainDF:
    popular_result[row.movieId] += 1
sorted_popular = sorted(popular_result.items(), key=operator.itemgetter(1))
sorted_popular.reverse()

In [164]:
sorted_popular

[(2458466, 356),
 (566845, 304),
 (2358174, 253),
 (2716671, 228),
 (2332760, 227),
 (2442067, 216),
 (1789325, 203),
 (2570279, 193),
 (337046, 187),
 (2381049, 184),
 (2947204, 156),
 (2160922, 151),
 (1831094, 146),
 (2357628, 141),
 (1774946, 141),
 (543450, 140),
 (2157629, 134),
 (2081801, 131),
 (1995533, 129),
 (1844720, 127),
 (1905044, 118),
 (2018485, 118),
 (612667, 110),
 (2687881, 109),
 (2118917, 96),
 (648142, 94),
 (2465088, 93),
 (2455911, 89),
 (1688452, 88),
 (1436324, 81),
 (563278, 79),
 (2156647, 79),
 (1930047, 76),
 (2190418, 75),
 (209083, 74),
 (1580227, 73),
 (593467, 72),
 (439498, 72),
 (509302, 68),
 (1822381, 68),
 (619580, 68),
 (2016464, 67),
 (2238392, 67),
 (2130367, 67),
 (2140646, 67),
 (1431819, 67),
 (2064561, 66),
 (2297773, 66),
 (2468314, 65),
 (2195345, 65),
 (2567800, 64),
 (1983367, 63),
 (1397305, 63),
 (253947, 63),
 (683266, 63),
 (1244162, 63),
 (2377504, 61),
 (2015202, 60),
 (2505760, 60),
 (237718, 60),
 (2707586, 60),
 (564527, 60),

In [165]:
sorted_popular_numbers_only = []
i = 0
for each in sorted_popular:
    sorted_popular_numbers_only.append(each[0])
    i += 1
    if i > 50:
        break

In [166]:
sorted_popular_numbers_only

[2458466,
 566845,
 2358174,
 2716671,
 2332760,
 2442067,
 1789325,
 2570279,
 337046,
 2381049,
 2947204,
 2160922,
 1831094,
 2357628,
 1774946,
 543450,
 2157629,
 2081801,
 1995533,
 1844720,
 1905044,
 2018485,
 612667,
 2687881,
 2118917,
 648142,
 2465088,
 2455911,
 1688452,
 1436324,
 563278,
 2156647,
 1930047,
 2190418,
 209083,
 1580227,
 593467,
 439498,
 509302,
 1822381,
 619580,
 2016464,
 2238392,
 2130367,
 2140646,
 1431819,
 2064561,
 2297773,
 2468314,
 2195345,
 2567800]

In [201]:
total_confidence_score = 0
for key, values in topDict.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:1] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            confidence_score += confidence
    total_confidence_score += confidence_score/len(values)
total_confidence_score/len(topDict)

0.5931628216062148

In [202]:
total_confidence_score = 0
k=15
topDict = dict()
for key, value in result.items():
    if len(value) >= k:
        topDict[key]=value[:k]
    else:
        topDict[key]=value
for key, values in topDict.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:k] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            confidence_score += confidence
    total_confidence_score += confidence_score/len(values)
total_confidence_score/len(topDict)

0.3148777813980439

In [203]:
uniq = set()
for key, value in topDict.items():
    for each in value:
        uniq.add(each[0])
len(uniq)

254082

In [175]:
total_confidence_score = 0
for key, values in result.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:50] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            #print(value)
            confidence_score += 1
    total_confidence_score += confidence_score/len(values)
total_confidence_score/len(result)

0.14395779185897498

In [177]:
total_confidence_score = 0
for key, values in result.items():
    confidence_score = 0
    for value, confidence in values:
        if value not in sorted_popular_numbers_only[:50] and (value in trainDict[key] or value in validDict[key] or value in testDict[key]):
            #print(value)
            confidence_score += confidence
    total_confidence_score += confidence_score/len(values)
total_confidence_score/len(result)

0.12933555223455978

In [184]:
k=50
precisions, recalls = [], []
for key in validDict.keys():
    n_rel = len(validDict[key])
    if n_rel == 0:
        continue
    n_rec_k = len(sorted_popular_numbers_only[:k])
    n_rel_and_rec_k = 0
    for value in sorted_popular_numbers_only[:k]:
        if value in validDict[key]:
            n_rel_and_rec_k += 1
    precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
    recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )
precision = np.sum(precisions)/len(precisions)
recall = np.sum(recalls)/len(recalls)
print("precision at k={}: ".format(k),precision)
print("recall at k={}: ".format(k),recall)
print("f1 score at k={}".format(k),2*(precision*recall)/(precision+recall))

precision at k=50:  0.0015578654373070064
recall at k=50:  0.03284676972742488
f1 score at k=50 0.002974648447252969


In [185]:
for k in [45, 40, 35, 30, 25, 20, 15, 10, 5,4,3,2,1]:
    precisions, recalls = [], []
    for key in validDict.keys():
        n_rel = len(validDict[key])
        if n_rel == 0:
            continue
        n_rec_k = len(sorted_popular_numbers_only[:k])
        n_rel_and_rec_k = 0
        for value in sorted_popular_numbers_only[:k]:
            if value in validDict[key]:
                n_rel_and_rec_k += 1
        precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
        recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )
    precision = np.sum(precisions)/len(precisions)
    recall = np.sum(recalls)/len(recalls)
    print("precision at k={}: ".format(k),precision)
    print("recall at k={}: ".format(k),recall)
    print("f1 score at k={}".format(k),2*(precision*recall)/(precision+recall))

precision at k=45:  0.0016217496146170092
recall at k=45:  0.030568678189558628
f1 score at k=45 0.00308009215502488
precision at k=40:  0.0017200887226547213
recall at k=40:  0.028968830736625456
f1 score at k=40 0.0032473583258399723
precision at k=35:  0.0018378264192207565
recall at k=35:  0.02699241517781526
f1 score at k=35 0.0034413429083066903
precision at k=30:  0.001932472201684571
recall at k=30:  0.024329674136545965
f1 score at k=30 0.003580546566102713
precision at k=25:  0.002068455616926891
recall at k=25:  0.021569361991855968
f1 score at k=25 0.0037749058482460444
precision at k=20:  0.002211542643413213
recall at k=20:  0.018602839249749353
f1 score at k=20 0.003953129379536981
precision at k=15:  0.0024297249887646967
recall at k=15:  0.015512676476147458
f1 score at k=15 0.004201392745606174
precision at k=10:  0.0029182794763623716
recall at k=10:  0.0121348541023263
f1 score at k=10 0.004705053003130994
precision at k=5:  0.003766363675901361
recall at k=5:  0.00

In [173]:
for k in [50, 45, 40, 35, 30, 25, 20, 15, 10, 5,4,3,2,1]:
    precisions, recalls = [], []
    for key in testDict.keys():
        n_rel = len(testDict[key])
        n_rec_k = len(sorted_popular_numbers_only[:k])
        n_rel_and_rec_k = 0
        for value in sorted_popular_numbers_only[:k]:
            if value in testDict[key]:
                n_rel_and_rec_k += 1
        precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1 )
        recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 1 )
    precision = np.sum(precisions)/len(precisions)
    recall = np.sum(recalls)/len(recalls)
    print("precision at k={}: ".format(k),precision)
    print("recall at k={}: ".format(k),recall)
    print("f1 score at k={}".format(k),2*(precision*recall)/(precision+recall))

precision at k=50:  0.0007707686417967588
recall at k=50:  0.5296893865396867
f1 score at k=50 0.0015392974007545363
precision at k=45:  0.0008006389987758825
recall at k=45:  0.5285021174964902
f1 score at k=45 0.0015988558571094769
precision at k=40:  0.0008443574801139989
recall at k=40:  0.5275553887191965
f1 score at k=40 0.0016860164746236707
precision at k=35:  0.0009029976361636577
recall at k=35:  0.5265661116797897
f1 score at k=35 0.0018029035093538415
precision at k=30:  0.000974095027436301
recall at k=30:  0.525429412407508
f1 score at k=30 0.0019445849834432729
precision at k=25:  0.0010276915223956782
recall at k=25:  0.5239388673078664
f1 score at k=25 0.0020513593604349384
precision at k=20:  0.0010889446594921093
recall at k=20:  0.5223021941070833
f1 score at k=20 0.0021733580979389753
precision at k=15:  0.0011839437378592596
recall at k=15:  0.5206634577034697
f1 score at k=15 0.0023625153199099648
precision at k=10:  0.001442000935811817
recall at k=10:  0.519091

In [178]:
precision

0.00246714024416181

In [182]:

for k in [50, 45, 40, 35, 30, 25, 20, 15, 10, 5,4,3,2,1]:
    precisions, recalls = [], []
    for key in testDict.keys():
        n_rel = len(testDict[key])
        if n_rel == 0:
            continue
        n_rec_k = len(sorted_popular_numbers_only[:k])
        n_rel_and_rec_k = 0
        for value in sorted_popular_numbers_only[:k]:
            if value in testDict[key]:
                n_rel_and_rec_k += 1
        precisions.append( n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0 )
        recalls.append( n_rel_and_rec_k / n_rel if n_rel != 0 else 0)
    precision = np.sum(precisions)/len(precisions)
    recall = np.sum(recalls)/len(recalls)
    print("precision at k={}: ".format(k),precision)
    print("recall at k={}: ".format(k),recall)
    print("f1 score at k={}".format(k),2*(precision*recall)/(precision+recall))

precision at k=50:  0.001583708429838745
recall at k=50:  0.03364661872669626
f1 score at k=50 0.0030250320115551214
precision at k=45:  0.0016450834437986472
recall at k=45:  0.03120712146353077
f1 score at k=45 0.0031254108509967417
precision at k=40:  0.001734912380369707
recall at k=40:  0.029261865437188346
f1 score at k=40 0.0032756161249079103
precision at k=35:  0.001855401077531043
recall at k=35:  0.027229184939053148
f1 score at k=35 0.0034740779220583623
precision at k=30:  0.0020014858191670674
recall at k=30:  0.024893594046943624
f1 score at k=30 0.003705077339133738
precision at k=25:  0.0021116112397849933
recall at k=25:  0.021830951495925423
f1 score at k=25 0.003850755916386602
precision at k=20:  0.002237468863348337
recall at k=20:  0.018468057620366435
f1 score at k=20 0.003991369543256296
precision at k=15:  0.002432664714708153
recall at k=15:  0.01510092445491137
f1 score at k=15 0.004190298486585733
precision at k=10:  0.0029628982213870565
recall at k=10:  0

In [189]:
list(result.items())[0][1]

[(3014656,
  [[2844431, 0.89999855],
   [1414361, 0.89999855],
   [2353240, 0.35424072],
   [810284, 0.35424072],
   [2373590, 0.35424072],
   [1308795, 0.35424072],
   [715905, 0.35424072],
   [1700713, 0.35424072],
   [2524756, 0.35424072],
   [1169490, 0.35424072],
   [1608379, 0.35121718],
   [883322, 0.34921202],
   [1191954, 0.3426835],
   [873556, 0.3426835],
   [1935004, 0.3426835],
   [775015, 0.3426835],
   [3022366, 0.3426835],
   [1658966, 0.34053558],
   [2832615, 0.34025428],
   [2319916, 0.34025428],
   [1744489, 0.34025428],
   [1959851, 0.34025428],
   [746409, 0.34025428],
   [2570870, 0.34025428],
   [1278931, 0.33620322],
   [1977474, 0.33620322],
   [1560164, 0.3354145],
   [1009760, 0.3354145],
   [535838, 0.33533752],
   [2369319, 0.33430693],
   [691882, 0.33209652],
   [3044867, 0.33209652],
   [2953581, 0.33209652],
   [2014824, 0.32461175],
   [1125119, 0.32414213],
   [2837517, 0.32414213],
   [631282, 0.32414213],
   [47828, 0.32414213],
   [1331911, 0.3241

In [190]:
reverse[3014656]

'a6987c73-0a2e-4382-a611-99a5d1e5a6f8'

In [194]:
t=1
print("paper: ",reverse[list(result.items())[t][0]])
for each in list(result.items())[t][1]:
    print(reverse[each[0]])

paper:  a6ae053d-ffcb-4fb9-9202-94eea961fbb4
32d10532-ba7e-4265-85ad-27991a1dcae3
48959853-1b8d-4a0d-a7ef-5e572797cb63
d62affd2-6bf4-45ce-bbb1-cfd5add1c369
a6ea33d7-0f41-4c42-838c-47e0d39bf811
0fd5f0f8-7f94-45ba-bf46-7e94abd016ce
717b9dfd-ae2f-4572-b2fa-07cf91fa680d
8c84c945-7a07-4176-9ce8-89d1f88401bb
2b131eee-476a-49dd-ad95-e8e5bbe2e77d
575dd986-a780-479f-8ff2-68e21561e01f
e54e8125-019a-4ddb-b033-257de5442922
ad7fbc28-919a-4848-8f31-f812d6e56a4a
d880ba87-9678-40ca-8c9f-717446f5c845
aa5428a6-5677-4878-a902-665776305915
a662e3f9-21cc-4dfc-976f-2bc8d3daaeed
41c79a56-e196-4420-b834-56f78ba35ac6
fc20a41e-c306-4f16-a751-56b57fc82b00
35647861-70c7-4022-b158-cd69447c9463
55455742-6b1f-41cd-9c8e-382c6364492b
8098bf75-533c-474d-97a8-1dc58bb885bb
7c6ac855-65a7-4008-b616-ade653096d9b
7c873d98-e156-4a35-a574-93a158f755ab
465047d5-ec82-435d-bf9e-9160741d8696
9806bcb5-d91b-4293-8af0-0bc1b11a6844
a084211a-5cf5-4321-924a-39d70e297432
405af087-ee43-49c1-81cb-69bbef8a6fe6
d9a8707e-d457-40c7-92e2-5c5702

In [195]:
t=2
print("paper: ",reverse[list(result.items())[t][0]])
for each in list(result.items())[t][1]:
    print(reverse[each[0]])

paper:  a6ff4e5f-d08d-4881-be7d-c631d1e61270
37a59214-5457-4e0c-8efa-c1cd2ebdafa4
1d0bce70-20e9-4253-b4a5-11a7c7aecf54
bdf5cfe4-02b7-46f3-9da3-62075f312d4d
2aa96c88-cc2d-4f5f-ae4d-45048142d518
952de3cd-b53a-4ace-a07f-eb4fc5128930
07cd3e4e-9253-41fd-964c-60b06ecc3c51
204a7a40-45ee-41a9-a0d2-16bb3dafe583
842fa636-4ffd-48e2-8713-8ac338cc73e6
c692f754-e6c1-4161-b457-5141375ea5de
62407d5f-621c-4fed-8f0b-ff8271dc0d14
21560d7b-80aa-40cc-8757-ddd4334a4b9c
88567d87-33a0-48c1-8ad9-7316814febad
f806d868-dec8-4451-8ab3-92ee155a1ad2
75634bd6-dd25-4d7b-a439-da2349e4e352
17982db5-8a9b-4795-b6f8-9b35044a7d42
0b0603b2-61ad-4a98-97ab-d963edca5234
35687fac-8ac7-4bd7-88a2-ebefae3a5022
487b0692-4adc-400f-a6ba-ab865d94ce45
7bc5788e-dbbd-48aa-9c2f-0ed38097f1d0
b92c9689-f804-440f-b89e-954cbe4821c7
0d1adb86-310d-47c2-9c8f-ba2b5fb4e70e
349d2c49-ea0c-4cf6-b05a-f8ca5d363164
e7e32aec-a66a-49cc-9135-a45d127a5ab4
8a64ddea-330e-4125-9d94-32ac412664fc
b6e42edf-d494-4cd0-b1f4-02916980f0f2
5e32955b-ce18-40a2-b2fe-46f9a1

In [198]:
refd = ["12d6aa75-3066-4e5f-a73d-f0d56c9d99f5", "65da1dc3-46fb-426d-a78f-30490ee43abb", "68603fc1-250e-4956-a71c-f666a627dfa7", "78b0ba4b-b8a0-4689-9972-cabab721ab40", "7ec5f06e-2fe3-495a-84a0-94fcfe08bb7b", "89f10062-acf1-4171-b882-f3222c3a357e", "96b45b5b-4ab5-468f-80a4-1ed1923cd481", "d3045079-6d4f-4c42-bfad-ce14b9bd69e7", "db807124-0169-4fdf-83b7-3157739d2c07", "ea8cd3d8-17ae-4a1e-8f83-1609469087af", "fd4c2dc1-317b-41f0-8469-2896b2768fbb"]

In [200]:
t=4
print("paper: ",reverse[list(result.items())[t][0]])
for each in list(result.items())[t][1]:
    if reverse[each[0]] not in refd:
        print(reverse[each[0]])
    else:
        print("already cited!")

paper:  a7cd1add-2988-4759-a08c-7a7646053468
6f8f9bc0-3c9a-4583-8819-2bd39263bd15
c1b6b493-01ef-420f-be44-7bacfe34e846
4773f066-c0ab-4ed4-9a02-7fe97871fea8
2f00ece1-1fda-4ada-bc3c-2a54ca68166c
c36b336c-4672-43f0-bdf5-022ab40fbc1a
feabe03b-938e-460b-be0a-97709571ed2e
f6376a8f-0cdd-4a55-98c4-a2dbe0a0301a
dad07d3b-7393-4ce0-a74e-aa0f4b541564
19491d37-876d-4a4d-95a9-f119494be93b
618a8daf-d2c7-480a-a6e6-ab7efca2702c
9fe917ad-113d-4d9a-99e3-ec0283a53fab
4aee6369-a9f1-47ed-88c9-b453c30eb831
b889d6ec-330d-406f-87b6-ea34804fadfd
4cbd7765-c47a-4004-a5f8-c2da7c7d1c7b
c0f1cb2b-e1ad-4ae2-abc0-c63b3389fef4
d9752a5a-1603-45cc-9a21-7997750d429f
e5f5cacc-f6da-458e-b8cb-bcfbd753e611
already cited!
14b24b9b-d458-4369-b2f5-f39c934be7d3
2fdbff7f-f7ab-4fd0-8211-898b47841a28
50dd56db-151d-4d62-8576-65f0ef6f381b
eacc9572-d663-457e-ae19-b32fc324fbb4
50b1abc6-dedb-4f55-978b-396b2e7a62c4
01b486c4-8955-403b-a0c6-1de74298b215
e407acd7-dfcf-4ee8-9140-6726c01abf4e
d28acb36-5766-4c1e-8d57-a55c2630bd90
2d45e773-5091-4